# Setup
- pip uninstall spacy 
- pip uninstall neuralcoref
- pip install spacy==2.1.0 
- pip install neuralcoref --no-binary neuralcoref
- python3 -m spacy download en
- USE MODEL : en_core_web_sm

In [ ]:
!pip install spacy==2.1.0

In [ ]:
!pip install neuralcoref --no-binary neuralcoref

In [ ]:
!python3 -m spacy download en

In [ ]:
from __future__ import print_function
import spacy
import neuralcoref
import os, sys
import re
import json
import array as arr
import uuid
from spacy import displacy
from spacy.matcher import Matcher
from json import JSONEncoder
import xml.etree.ElementTree as gfg
from xml.etree import ElementTree
import os.path
from nltk.tree import ParentedTree

# Read File

In [ ]:
text = "The Vacation Request Process starts when an employee of the organization submits a vacation request." \
"Once the requirement is registered, the request is received by the immediate supervisor; the supervisor must approve or reject the request." \
"If the request is rejected the application is returned to the applicant/employee who can review the rejection reasons." \
"If the request is approved a notification is generated to the Human Resources representative, who must complete the respective administrative procedures."

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Resolve sentence from neuralcoref


In [ ]:
import neuralcoref

doc = nlp(text)
resolved_text = doc._.coref_resolved
sentences = [sent.string.strip() for sent in nlp(resolved_text).sents]
print(sentences)

for index, text in enumerate(sentences):
  print ("AT INDEX", index, "=", text)
  #getNounPhrase(text)

['The Vacation Request Process starts when an employee of the organization submits a vacation request.', 'Once the requirement is registered, a vacation request is received by the immediate supervisor; the immediate supervisor must approve or reject a vacation request.', 'If a vacation request is rejected the application is returned to the applicant/employee who can review the rejection reasons.', 'If a vacation request is approved a notification is generated to the Human Resources representative, who must complete the respective administrative procedures.']
AT INDEX 0 = The Vacation Request Process starts when an employee of the organization submits a vacation request.
AT INDEX 1 = Once the requirement is registered, a vacation request is received by the immediate supervisor; the immediate supervisor must approve or reject a vacation request.
AT INDEX 2 = If a vacation request is rejected the application is returned to the applicant/employee who can review the rejection reasons.
AT IN

In [ ]:
import os, sys
from nltk import Tree

#"The Vacation Request Process starts when an employee of the organization submits a vacation request."
text = "(TOP (S (NP (DT The) (NNP Vacation) (NNP Request) (NNP Process)) (VP (VBZ starts) (SBAR (WHADVP (WRB when)) (S (NP (NP (DT an) (NN employee)) (PP (IN of) (NP (DT the) (NN organization)))) (VP (VBZ submits) (NP (DT a) (NN vacation) (NN request.))))))))"
parsetree = Tree.fromstring(text)
parsetree.pretty_print()

                       TOP                                                                                             
                        |                                                                                               
                        S                                                                                              
        ________________|___________________                                                                            
       |                                    VP                                                                         
       |                        ____________|____________________                                                       
       |                       |                                SBAR                                                   
       |                       |       __________________________|____________                                          
       |                       |    

In [ ]:
import os, sys
from nltk import Tree

#Once the requirement is registered, the request is received by the immediate supervisor; the supervisor must approve or reject the request.
text = "(TOP (S (SBAR (RB Once) (S (NP (DT the) (NN requirement)) (VP (VBZ is) (VP (VBN registered,) (SBAR (S (NP (DT the) (NN request)) (VP (VBZ is) (VP (VBN received) (PP (IN by) (NP (DT the) (JJ immediate) (NN supervisor;))))))))))) (NP (DT the) (NN supervisor)) (VP (MD must) (VP (VP (VB approve)) (CC or) (VP (VB reject) (NP (DT the) (NN request.)))))))"
nltk_tree = Tree.fromstring(text)
nltk_tree.pretty_print()

                                                                           TOP                                                                                      
                                                                            |                                                                                        
                                                                            S                                                                                       
       _____________________________________________________________________|_____________________________________________________________                           
     SBAR                                                                                                     |                           |                         
  ____|____________                                                                                           |                           |                          
 |     

In [ ]:
import os, sys
from nltk import Tree

#If the request is rejected the application is returned to the applicant/employee who can review the rejection reasons.
text = "(TOP (SBAR (IN If) (S (NP (DT the) (NN request)) (VP (VBZ is) (VP (VBN rejected) (SBAR (S (NP (DT the) (NN application)) (VP (VBZ is) (VP (VBN returned) (PP (TO to) (NP (NP (DT the) (NN applicant/employee)) (SBAR (WHNP (WP who)) (S (VP (MD can) (VP (VB review) (NP (DT the) (NN rejection))))))))))))))) (. reasons.)))"
parsetree = Tree.fromstring(text)
parsetree.pretty_print()

                                                           TOP                                                                               
                                                            |                                                                                 
                                                           SBAR                                                                              
  __________________________________________________________|___________________________________________________________________________      
 |             S                                                                                                                        |    
 |        _____|___________                                                                                                             |     
 |       |                 VP                                                                                                           |    
 | 

In [ ]:
import os, sys
from nltk import Tree

#If the request is approved a notification is generated to the Human Resources representative, who must complete the respective administrative procedures.
text = "(TOP (SBAR (IN If) (S (NP (DT the) (NN request)) (VP (VBZ is) (VP (VBN approved) (SBAR (S (NP (DT a) (NN notification)) (VP (VBZ is) (VP (VBN generated) (PP (TO to) (NP (NP (DT the) (NNP Human) (NNPS Resources)) (, representative,) (SBAR (WHNP (WP who)) (S (VP (MD must) (VP (VB complete) (NP (DT the) (JJ respective) (JJ administrative) (NN procedures.)))))))))))))))))"
parsetree = Tree.fromstring(text)
parsetree.pretty_print()

    TOP                                                                                                                                                                  
     |                                                                                                                                                                    
    SBAR                                                                                                                                                                 
  ___|__________                                                                                                                                                          
 |              S                                                                                                                                                        
 |         _____|___________                                                                                                                        

In [ ]:
def all_label(nltk_tree):
    for s in nltk_tree.subtrees():
        if s.label() != 'S' and s.label() != 'ROOT' and s.height() > 2:
            print(s.label() + ': ', end='')
            print(' '.join(s.leaves()).replace('_', ' '))

def get_text_label(nltk_tree, get_label='NP'):
    for s in nltk_tree.subtrees():
        if s.label() == get_label:
            print(' '.join(s.leaves()).replace('_', ' '))

def get_text_tag(nltk_tree, get_tag='N'):
    for s in nltk_tree.subtrees(lambda t: t.height() == 2):
        if s.label() == get_tag:
            print(s.leaves()[0])

In [ ]:
all_label(nltk_tree)

TOP: Once the requirement is registered, the request is received by the immediate supervisor; the supervisor must approve or reject the request.
SBAR: Once the requirement is registered, the request is received by the immediate supervisor;
NP: the requirement
VP: is registered, the request is received by the immediate supervisor;
VP: registered, the request is received by the immediate supervisor;
SBAR: the request is received by the immediate supervisor;
NP: the request
VP: is received by the immediate supervisor;
VP: received by the immediate supervisor;
PP: by the immediate supervisor;
NP: the immediate supervisor;
NP: the supervisor
VP: must approve or reject the request.
VP: approve or reject the request.
VP: approve
VP: reject the request.
NP: the request.


In [ ]:
get_text_label(nltk_tree, 'NP')

the requirement
the request
the immediate supervisor;
the supervisor
the request.


In [ ]:
get_text_tag(nltk_tree, 'N')

In [ ]:
get_text_label(nltk_tree, 'VP')

is registered, the request is received by the immediate supervisor;
registered, the request is received by the immediate supervisor;
is received by the immediate supervisor;
received by the immediate supervisor;
must approve or reject the request.
approve or reject the request.
approve
reject the request.


In [ ]:
get_text_tag(nltk_tree, 'VBD')

In [ ]:
get_text_label(nltk_tree, 'PP')

by the immediate supervisor;


In [ ]:
get_text_tag(nltk_tree, 'IN')

by


In [ ]:
#====================================================================================
#                                  SIGNAL WORD
#====================================================================================
SIGNAL_WORD = ['Once', 'Then', 'then', 'When', 'After', 'Afterwards', 'As soon as', 'Subsequently',
               'Upon', 'After that', 'Likewise', 'after which', 'Immediately after', 'In addition to',
               'In the following', 'Moreover', 'Thereafter', 'Therefore']
def check_signal_word_pattern(doc):
    for i in range(len(doc)-1):
        if doc[i].dep_ == 'mark' and doc[i].text in SIGNAL_WORD:
            return True
        return False

In [ ]:
list_sentence = []
list_sentence_active = []
list_prep_word = []

def condition_mark(token):
    if token.dep_ == "mark":
        if token.text in ["If"]:
            return True
    return False

def find_dependency_active_sentence(token):
    if token.dep_ == "nsubj":
        word_left = " ".join([token.text for token in token.lefts])
        list_sentence_active.append(word_left + " " + token.text + "[nsubj]")
    elif token.dep_ in ["ROOT"] and token.tag_ in ["VBZ"]:
        list_sentence_active.append(token.text + "[verb]")
    elif token.dep_ in ["dobj","ccomp"]:
        word_left = " ".join([token.text for token in token.lefts])
        list_sentence_active.append(word_left + " " + token.text + "[dobj]")

def find_dependency_passive_sentence(token):
    if token.dep_ == "nsubjpass":
        list_sentence.append(token.text + "[nsubjpass]")

    elif token.dep_ == "ROOT" and token.tag_ in ["VBN","VBD"]:
        word_right = " ".join([token.text for token in token.rights])
        list_sentence.append(token.text + "[verb]" + word_right)

    elif token.dep_ == "aux":
        if token.text in ["shall","should","will","could"]:
            list_sentence.append(token.text + "[aux]")

    elif token.dep_ == "prep":
        word_right = " ".join([token.text for token in token.rights])
        list_sentence.append(token.text + "[prep]" + word_right + " ")
        list_prep_word.extend(word_right)

    elif token.dep_ == "pobj":
        if list_prep_word is None:
            word_left = " ".join([token.text for token in token.lefts])
            list_sentence.append(word_left + " " + token.text + "[pobj]")

#====================================================================================
def active_sentence_match_pattern(text):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)
    doc = nlp(text)
    sents = list(doc.sents)
    print("Number of Sentences = ",len(sents))
    passive_rule = [{'DEP':'nsubj'},{'POS':'VERB'}]
    matcher.add('Passive',None,passive_rule)
    matches = matcher(doc)
    print("Match :", len(matches))

    for match in matches:
        match_id, start, end = match
        span = doc[start:end]
        print(match, span.text)

def passive_sentence_match_pattern(text):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)
    doc = nlp(text)
    sents = list(doc.sents)
    print("Number of Sentences = ",len(sents))
    passive_rule = [{'DEP':'nsubjpass'},{'DEP':'aux','OP':'*'},{'DEP':'auxpass'},{'TAG':'VBN'}]
    matcher.add('Passive',None,passive_rule)
    matches = matcher(doc)
    print("Match :", len(matches))

    for match in matches:
        match_id, start, end = match
        span = doc[start:end]
        print(match, span.text)

In [ ]:
text = "Once the requirement is registered, the request is received by the immediate supervisor; the supervisor must approve or reject the request."
passive_sentence_match_pattern(text)

Number of Sentences =  1
Match : 2
(3889985946448656432, 2, 5) requirement is registered
(3889985946448656432, 7, 10) request is received


In [ ]:
text = "Once the requirement is registered, the request is received by the immediate supervisor; the supervisor must approve or reject the request."
active_sentence_match_pattern(text)

Number of Sentences =  1
Match : 1
(3889985946448656432, 16, 18) supervisor must


In [ ]:
!pip install stanfordnlp

In [ ]:
from nltk.tag.stanford import StanfordNERTagger
from nltk.tokenize import word_tokenize
import nltk

!wget 'https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip'
!unzip stanford-ner-2018-10-16.zip

nltk.download('punkt')
nltk.parse

st = StanfordNERTagger('/content/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/content/stanford-ner-2018-10-16/stanford-ner.jar',
                       encoding='utf-8')

text = 'Once the requirement is registered, the request is received by the immediate supervisor; the supervisor must approve or reject the request.'

tokenized_text = word_tokenize(text)

In [ ]:
classified_text = st.tag(tokenized_text)
print(classified_text)

[('Once', 'O'), ('the', 'O'), ('requirement', 'O'), ('is', 'O'), ('registered', 'O'), (',', 'O'), ('the', 'O'), ('request', 'O'), ('is', 'O'), ('received', 'O'), ('by', 'O'), ('the', 'O'), ('immediate', 'O'), ('supervisor', 'O'), (';', 'O'), ('the', 'O'), ('supervisor', 'O'), ('must', 'O'), ('approve', 'O'), ('or', 'O'), ('reject', 'O'), ('the', 'O'), ('request', 'O'), ('.', 'O')]


In [ ]:
import nltk
nltk.download('book')
nltk.download('tests')
nltk.download('all-corpora')# not recommended as it download huge amount of data.
nltk.download('punkt')
nltk.download('popular')

In [ ]:
!unzip stanford-parser-3.5.1-models.jar

In [ ]:
import nltk
from nltk.tree import *
from nltk.parse import stanford
import nltk.data
import nltk.draw

class SVO(object):
    """
    Class Methods to Extract Subject Verb Object Tuples from a Sentence
    """

    def __init__(self):
        """
        Initialize the SVO Methods
        """
        self.noun_types = ["NN", "NNP", "NNPS", "NNS", "PRP"]
        self.verb_types = ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]
        self.adjective_types = ["JJ", "JJR", "JJS"]
        self.pred_verb_phrase_siblings = None
        self.parser = stanford.StanfordParser(
            '/content/stanford-parser-3.5.1-models.jar',
            '/content/stanford-parser.jar'
        )
        self.sent_detector = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

    def get_attributes(self, node, parent_node, parent_node_siblings):
        """
        returns the Attributes for a Node
        """

    def get_subject(self, sub_tree):
        """
        Returns the Subject and all attributes for a subject, sub_tree is a Noun Phrase

        :param sub_tree:
        :return:
        """
        sub_nodes = sub_tree.subtrees()
        print("SUBJECT sub_tree.label() ->", sub_tree.label())
        sub_nodes = [each for each in sub_nodes if each.pos()]
        subject = None

        for each in sub_nodes:
            print("each.label() ->", each.label())
            if each.label() in self.noun_types:
                subject = each.leaves()
                print("GET SUBJECT ->", subject)
                break

        return {'subject': subject}

    def get_object(self, sub_tree):
        """
        Returns an Object with all attributes of an object
        """
        siblings = self.pred_verb_phrase_siblings
        Object = None
        for each_tree in sub_tree:
            if each_tree.label() in ["NP", "PP"]:
                sub_nodes = each_tree.subtrees()
                sub_nodes = [each for each in sub_nodes if each.pos()]

                for each in sub_nodes:
                    if each.label() in self.noun_types:
                        Object = each.leaves()
                        break
                break
            else:
                sub_nodes = each_tree.subtrees()
                sub_nodes = [each for each in sub_nodes if each.pos()]
                for each in sub_nodes:
                    if each.label() in self.adjective_types:
                        Object = each.leaves()
                        break
                # Get first noun in the tree
        self.pred_verb_phrase_siblings = None
        return {'object': Object}

    def get_predicate(self, sub_tree):
        """
        Returns the Verb along with its attributes, Also returns a Verb Phrase

        :param sub_tree:
        :return:
        """

        sub_nodes = sub_tree.subtrees()
        print("PREDICATE sub_nodes.label() ->", sub_tree.label())
        sub_nodes = [each for each in sub_nodes if each.pos()]
        predicate = None
        sub_tree = ParentedTree.convert(sub_tree)
        for each in sub_nodes:
            if each.label() in self.verb_types:
                sub_tree = each
                predicate = each.leaves()

        # get all predicate_verb_phrase_siblings to be able to get the object
        sub_tree = ParentedTree.convert(sub_tree)
        if predicate:
            pred_verb_phrase_siblings = self.tree_root.subtrees()
            pred_verb_phrase_siblings = [each for each in pred_verb_phrase_siblings if
                                         each.label() in ["NP", "PP", "ADJP", "ADVP"]]
            self.pred_verb_phrase_siblings = pred_verb_phrase_siblings

        return {'predicate': predicate}

    def process_parse_tree(self, parse_tree):
        """
        Returns the Subject-Verb-Object Representation of a Parse Tree.
        Can Vary depending on number of 'sub-sentences' in a Parse Tree
        """
        self.tree_root = parse_tree
        # Step 1 - Extract all the parse trees that start with 'S'
        output_list = []
        output_dict = {}

        for idx, subtree in enumerate(parse_tree[0].subtrees()):
            subject = None
            predicate = None
            Object = None
            if subtree.label() in ["S", "SQ", "SBAR", "SBARQ", "SINV", "FRAG"]:
                children_list = subtree
                children_values = [each_child.label() for each_child in children_list]
                children_dict = dict(zip(children_values, children_list))

                # Extract Subject, Verb-Phrase, Objects from Sentence sub-trees
                if children_dict.get("NP") is not None:
                    subject = self.get_subject(children_dict["NP"])

                if children_dict.get("VP") is not None:
                    # Extract Verb and Object
                    # i+=1
                    # """
                    # if i==1:
                    #    pdb.set_trace()
                    # """
                    predicate = self.get_predicate(children_dict["VP"])
                    Object = self.get_object(children_dict["VP"])

                try:
                    if subject['subject'] and predicate['predicate'] and Object['object']:
                        output_dict['subject'] = subject['subject']
                        output_dict['predicate'] = predicate['predicate']
                        output_dict['object'] = Object['object']
                        output_list.append(output_dict)
                except Exception as e:
                    print(e)
                    continue

        return output_list

    def traverse(self, t):
        try:
            t.label()
        except AttributeError:
            print(t)
        else:
            # Now we know that t.node is defined
            print('(', t.label())
            for child in t:
                self.traverse(child)

            print(')')

    def sentence_split(self, text):
        """
        returns the Parse Tree of a Sample
        """
        sentences = self.sent_detector.tokenize(text)
        return sentences

    def get_parse_tree(self, sentence):
        """
        returns the Parse Tree of a Sample
        """
        parse_tree = self.parser.raw_parse(sentence)

        return parse_tree

    def List_To_Tree(self, lst):
        if (not isinstance(lst, basestring)):
            if (len(lst) == 2 and isinstance(lst[0], str) and isinstance(lst[1], str)):
                lst = Tree(str(lst[0]).split('+')[0], [str(lst[1])])
            elif (isinstance(lst[0], str) and not isinstance(lst[1], str)):
                lst = Tree(str(lst[0]).split('+')[0], map(self.List_To_Tree, lst[1: len(lst)]))
        return lst

svo = SVO()
sentence = "Once the requirement is registered, the request is received by the immediate supervisor; the supervisor must approve or reject the request."
sentences = svo.sentence_split(sentence)
val = []
for sent in sentences:
    root_tree = svo.get_parse_tree(sent)
    val.append(svo.process_parse_tree(next(root_tree)))
print(val)

'NoneType' object is not subscriptable
SUBJECT sub_tree.label() -> NP
each.label() -> NP
each.label() -> DT
each.label() -> NN
GET SUBJECT -> ['request']
PREDICATE sub_nodes.label() -> VP
'NoneType' object is not subscriptable
SUBJECT sub_tree.label() -> NP
each.label() -> NP
each.label() -> DT
each.label() -> NN
GET SUBJECT -> ['requirement']
PREDICATE sub_nodes.label() -> VP
SUBJECT sub_tree.label() -> NP
each.label() -> NP
each.label() -> DT
each.label() -> NN
GET SUBJECT -> ['supervisor']
PREDICATE sub_nodes.label() -> VP
[[{'subject': ['request'], 'predicate': ['received'], 'object': ['immediate']}]]


In [23]:
text = "The Support Officer updates all group calendars"

import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
import re 
import string 
import nltk 
import spacy 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 

pd.set_option('display.max_colwidth', 200)

In [ ]:
#==================================================================================
#                                 GET NOUN PHRASE
#==================================================================================
def getNounPhrase(text):
    noun_chunks_list = []
    doc = nlp(text)

    #Get Noun Phrase
    for np in doc.noun_chunks: 
        noun_chunks_list.append(str(np))
    
    verbs = [tok for tok in doc if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    for v in verbs:
        subs = getAllSubsLefts(v)
        dobj_right_word_list.append(getObjectRights(v))
    
    #==================================================================================
    #                       GET NOUN PHRASE RELATION WITH VERB
    #==================================================================================
    index = 0
    nounIndices = []
    for token in doc:
        if token.pos_ == 'NOUN':
            nounIndices.append(index)
        index = index + 1

    for idxValue in nounIndices:
        doc = nlp(text)
        span = doc[doc[idxValue].left_edge.i : doc[idxValue].right_edge.i+1]
        span.merge()

        for token in doc:
            if token.dep_ in SUBJECTS:
                nsubj_list.append(token.text)
            elif token.dep_ == 'dobj' or token.dep_ == 'pobj':
                dobj_list.append(token.text)
            elif token.dep_ == 'pobj' or token.pos_ == "PRON":
                pobj_list.append(token.text)

    #==================================================================================
    #                            GET NSUBJ, DOBJ, POBJ 
    #==================================================================================

    #Get Noun Phrase
    print("nsubj_list :", nsubj_list)
    print("dobj_list :", dobj_list)
    print("pobj_list :", pobj_list)

    #==================================================================================
    #                            หา NSUBJ, DOBJ, POBJ 
    #==================================================================================
    if not nsubj_list:
        print("กรณีหาประธานไม่ได้ เจอ Activity ที่เป็น Compound")
        all_subject = str(composite_activity_list[0]).replace('[','').replace(']','')
    else:
        all_subject = " ".join(set(nsubj_list).intersection(noun_chunks_list))
        # กรณีไม่เจอ Subject เลย
        if len(all_subject) == 0:
            all_subject = str(noun_chunks_list[0])
            print("หาประธานได้ =", all_subject)

    all_verb = str(verbs[0])

    if not dobj_list:
        print("ไม่มี object")
        all_object = str(dobj_right_word_list[0]).replace('[','').replace(']','')
    else:
        print("มี object จาก Noun Phrase")
        all_object = " ".join(set(dobj_list).intersection(noun_chunks_list))
    
    #==================================================================================
    #                             CREATE JSON OF SENTENCE
    #==================================================================================
    create_json_sentence(all_subject, all_verb, all_object)

def getAllSubsLefts(v):
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.dep_ is not "det"]
    #กรณีหาประธานไม่ได้ เจอ Activity ที่เป็น Compound
    getCompound(v)
    return subs

def getCompound(v):
    compound = [tok for tok in v.lefts if tok.dep_ == "compound"]
    composite_activity_list.append(compound)

def getObjectRights(v):
    object_sentence = [tok for tok in v.rights]
    return object_sentence

def getMultipleVerbRights(v):
    multiple_verb_sentence = [tok for tok in v.rights if tok.dep_ == "conj"]
    return multiple_verb_sentence

In [ ]:
#define the pattern 
pattern = [{'POS':'NOUN'}, 
           {'LOWER': 'such'}, 
           {'LOWER': 'as'}, 
           {'POS': 'PROPN'} #proper noun
          ]

In [ ]:
# Matcher class object 
matcher = Matcher(nlp.vocab) 
matcher.add("matching_1", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 

print(span.text)

countries such as Vietnam


In [ ]:
# Matcher class object
matcher = Matcher(nlp.vocab)

#define the pattern
pattern = [{'DEP':'amod', 'OP':"?"}, # adjectival modifier
           {'POS':'NOUN'},
           {'LOWER': 'such'},
           {'LOWER': 'as'},
           {'POS': 'PROPN'}]

matcher.add("matching_1", None, pattern)
matches = matcher(doc)

span = doc[matches[0][1]:matches[0][2]]
print(span.text)

developing countries such as Vietnam


In [ ]:
# Matcher class object 
matcher = Matcher(nlp.vocab) 

#define the pattern 
pattern = [{'DEP':'amod', 'OP':"?"}, 
           {'POS':'NOUN'}, 
           {'LOWER': 'and', 'OP':"?"}, 
           {'LOWER': 'or', 'OP':"?"}, 
           {'LOWER': 'other'}, 
           {'POS': 'NOUN'}] 
           
matcher.add("matching_1", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 
print(span.text)

car and other vehicles


In [ ]:
# Matcher class object 
matcher = Matcher(nlp.vocab) 

#define the pattern 
pattern = [{'DEP':'nummod','OP':"?"}, # numeric modifier 
           {'DEP':'amod','OP':"?"}, # adjectival modifier 
           {'POS':'NOUN'}, 
           {'IS_PUNCT': True}, 
           {'LOWER': 'including'}, 
           {'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}] 
                               
matcher.add("matching_1", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 
print(span.text)

Eight people, including two children


In [ ]:
# Matcher class object 
matcher = Matcher(nlp.vocab)

#define the pattern 
pattern = [{'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}, 
           {'IS_PUNCT':True}, 
           {'LOWER': 'especially'}, 
           {'DEP':'nummod','OP':"?"}, 
           {'DEP':'amod','OP':"?"}, 
           {'POS':'NOUN'}] 
           
matcher.add("matching_1", None, pattern) 

matches = matcher(doc) 
span = doc[matches[0][1]:matches[0][2]] 
print(span.text)

fruits, especially whole fruits


In [ ]:
def subtree_matcher(doc): 
  x = '' 
  y = '' 
  
  # iterate through all the tokens in the input sentence 
  for i,tok in enumerate(doc): 
    # extract subject 
    if tok.dep_.find("subjpass") == True: 
      y = tok.text 
      
    # extract object 
    if tok.dep_.endswith("obj") == True: 
      x = tok.text 
      
  return x,y

In [ ]:
text_2 = "Careem, a ride hailing major in middle east, was acquired by Uber." 

doc_2 = nlp(text_2) 
subtree_matcher(doc_2)

('Uber', 'Careem')

In [ ]:
text_3 = "Salesforce recently acquired Tableau." 
doc_3 = nlp(text_3) 
subtree_matcher(doc_3)

('Tableau', '')